# Индивидуальное задание 8

## Задача №1 классификатор k-NN

### Первый способ

In [132]:
import numpy as np
import pandas as pd
import io

from scipy.spatial import distance

In [133]:
data = '''id,X,Y,Class
1,51,43,1
2,22,87,1
3,10,55,0
4,90,17,1
5,59,65,0
6,13,44,0
7,73,50,1
8,75,24,1
9,31,19,0
10,55,93,0'''

df = pd.read_csv(io.StringIO(data), index_col='id')
df

,X,Y,Class
id,,,
1,51,43,1
2,22,87,1
3,10,55,0
4,90,17,1
5,59,65,0
6,13,44,0
7,73,50,1
8,75,24,1
9,31,19,0


In [134]:
z = np.array([34,28])
df['Euclidean'] = [np.round(distance.euclidean(obj, z), 3) for index, obj in df.iloc[:,:2].iterrows()]
df['Manhattan'] = [distance.cityblock(obj, z) for index, obj in df.iloc[:,:2].iterrows()]
df['Chebyshev'] = [distance.chebyshev(obj, z) for index, obj in df.iloc[:,:2].iterrows()]

df.sort_values(['Euclidean']).iloc[:3,:].style.highlight_min(['Euclidean', 'Manhattan', 'Chebyshev'], color = 'red', axis = 0)

,X,Y,Class,Euclidean,Manhattan,Chebyshev
id,,,,,,
9,31,19,0,9.487000,12,9
1,51,43,1,22.672000,32,17
6,13,44,0,26.401000,37,21


### Второй способ

In [139]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_euclidean = KNeighborsClassifier(n_neighbors=3, p=2) # , p = 2 - евклидову
knn_euclidean.fit(df[['X', 'Y']], df['Class'])

z = np.array([34, 28])
knn_euclidean.predict([z])
#knn_euclidean.predict_proba(z)
knn_euclidean.kneighbors([z])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


(array([[ 9.48683298, 22.6715681 , 26.40075756]]), array([[8, 0, 5]]))

In [142]:
knn_cityblock = KNeighborsClassifier(n_neighbors=3, p=1) # p = 1 - манхэттенская
knn_cityblock.fit(df[['X', 'Y']], df['Class'])
knn_cityblock.predict([z])
#knn_cityblock.predict_proba([z])
knn_cityblock.kneighbors([z])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


(array([[12., 32., 37.]]), array([[8, 0, 5]]))

## Задача №2 наивный байесовский классификатор

In [246]:
spam_letters = 12
not_spam_letters = 24

total_letters = spam_letters + not_spam_letters

spam_words = 76
not_spam_words = 185

spam_prob = spam_letters / total_letters
not_spam_prob = not_spam_letters / total_letters

ln_spam = np.log(spam_prob)
ln_not_spam = np.log(not_spam_prob)

word_list='''word,spam,not_spam
bonus,3,19
coupon,0,6
membership,3,3
unlimited,3,11
purchase,1,34
prize,6,3
investment,13,7
money,4,39
bill,7,53
free,36,10'''

df_words = pd.read_csv(io.StringIO(word_list), index_col='word')

alphabet_len = len(df_words.index.values)
alphabet_len

10

In [247]:

phrase = np.array('Coupon Prize Membership Refund Investment Purchase Online'.lower().split())
phrase

array(['coupon', 'prize', 'membership', 'refund', 'investment',
       'purchase', 'online'], dtype='<U10')

In [248]:
round(spam_prob, 3)

0.333

In [249]:
df_words.index.values

array(['bonus', 'coupon', 'membership', 'unlimited', 'purchase', 'prize',
       'investment', 'money', 'bill', 'free'], dtype=object)

In [250]:
new_words = [x for x in phrase if x not in df_words.index.values]
new_words

['refund', 'online']

In [251]:
for word in new_words:
  df_words.loc[word] = [0, 0]
df_words

,spam,not_spam
word,,
bonus,3,19
coupon,0,6
membership,3,3
unlimited,3,11
purchase,1,34
prize,6,3
investment,13,7
money,4,39
bill,7,53


In [252]:

r = len(new_words)
r

2

In [253]:
spam_word_probs = [((1 + df_words.loc[word].spam) / (alphabet_len + r + spam_words)) for word in phrase]

F_spam = ln_spam + sum(np.log(spam_word_probs))
np.round(F_spam, 3)

-25.776

In [254]:
not_spam_word_probs = [((1 + df_words.loc[word].not_spam) / (alphabet_len + r + not_spam_words)) for word in phrase]

F_not_spam = ln_not_spam + sum(np.log(not_spam_word_probs))
np.round(F_not_spam, 3)

-27.035

In [255]:
P_spam = 1 / (1 + np.exp(F_not_spam - F_spam))
np.round(P_spam, 3)

0.779

In [ ]:
from sklearn.naive_bayes import GaussianNB
